In [1]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from geopy.geocoders import Nominatim

In [2]:
# get the API key from environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    print("API key loaded successfully!")
else:
    print("API key not found in environment variables.")

API key loaded successfully!


In [47]:
geolocator = Nominatim(user_agent="ai-geolocation")
def get_coordinates(location: str) -> dict:
    """Get coordinates and bounding box for a location using Nominatim"""
    passed_location = location
    try:
        location = geolocator.geocode(location)
        if location:
            return {
                "passed_location": passed_location,
                "latitude": location.latitude,
                "longitude": location.longitude,
                "bbox": location.raw['boundingbox'],
                "address": location.address
            }
        return {"error": "Location not found"}
    except Exception as e:
        return {"error": str(e)}

# function for gemini
geocode_func = {
    "name": "get_coordinates",
    "description": "Get geographic coordinates and bounding box for a location string",
    "parameters": {
        "type": "OBJECT",
        "properties": {
            "location": {
                "type": "STRING",
                "description": "The location to geocode (e.g., 'Paris, France', 'Statue of Liberty')"
            }
        },
        "required": ["location"]
    }
}

In [81]:
# test get_coordinates
print(get_coordinates('ALLA CASCATA DEL TOCE IN VALTORMAZZA'))

{'error': 'Location not found'}


In [48]:
config = {
    'tools': [get_coordinates],
}

In [49]:
client = genai.Client(api_key=api_key)

In [87]:
locality_name = "Val Aurina"

In [88]:
# test with gemini thinking experimental with google maps (openstreetmaps) integration
# limitations: (see)
# - json output not supported yet 
# - thinking process not visible (yet?)
# awesome wins
# TORRE AMALIA 
prompt = """
You are an expert in georeferencing localities from herbaria labels

I'm gonna pass you the name of a locality
First , if needed format the string to make it more compatible with google maps:
- translating if necessary (some use latin words), 
- removing unuseful or misleading parts from the string
- add full words instead of abbreviations, for example mount is usually written as m. or mt.
Then retrive the coordinates use google maps wrapper get_coordinates

You MUST use get_coordinates tool to get the precise coordinates of the locality, the function returns a dictionary with some data
If the function doesn't find a locality returns "error": "Location not found" 
since it use google maps if you get error retry improving the string format and retry to call get_coordinates

when you get a good gereferentiation, return the raw result of get_coordinates along the thinking process
if you find no result say it please don't lie

locality: """ + locality_name + "'"

response = client.models.generate_content(
    # model='gemini-2.0-flash-thinking-exp-01-21', # maledetti di merda no function calling per lui
    model='gemini-2.0-flash',
    contents=prompt,
    config=config
)
print(response.text)

The `get_coordinates` tool found a result for "Val Aurina". The result includes the address, bounding box, latitude, and longitude.

Here's the raw result:
```json
{"result": {"address": "Ahrntal - Valle Aurina, Pustertal - Val Pusteria, Bolzano - Bozen, Trentino-Alto Adige/Südtirol, 39030, Italia", "bbox": ["46.9148896", "47.0619173", "11.8068780", "12.0775985"], "latitude": 46.98829205, "longitude": 11.943646067399136, "passed_location": "Val Aurina"}}
```

